In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("C:\\Users\\sanka\\Desktop\\JPMorganVirtualinternship\\Task 3 and 4_Loan_Data.csv")

In [3]:
import itertools

fico_scores = df['fico_score'].values
defaults = df['default'].values
fico_df = pd.DataFrame({'fico_score': fico_scores, 'default': defaults})

low_range = fico_df[fico_df['fico_score'] <= 600].sort_values('fico_score').reset_index(drop=True)
high_range = fico_df[fico_df['fico_score'] > 600].sort_values('fico_score').reset_index(drop=True)

def compute_log_likelihood(bucketed_data):
    """Compute total log-likelihood given a list of buckets (DataFrames)."""
    total_ll = 0
    for bucket in bucketed_data:
        ni = len(bucket)
        ki = bucket['default'].sum()
        if ni == 0:
            continue
        pi = (ki + 0.5) / (ni + 1)
        total_ll += ki * np.log(pi) + (ni - ki) * np.log(1 - pi)
    return total_ll

def find_best_buckets(fico_df, num_buckets):
    """Greedily find best boundaries to maximize log-likelihood."""
    percentiles = np.linspace(0, 100, num_buckets + 1)[1:-1]
    candidates = np.percentile(fico_df['fico_score'], percentiles).astype(int)

    best_ll = -np.inf
    best_boundaries = None

    for split in itertools.combinations(candidates, num_buckets - 1):
        boundaries = [fico_df['fico_score'].min()] + list(split) + [fico_df['fico_score'].max()]
        bucketed = [
            fico_df[(fico_df['fico_score'] > boundaries[i]) & (fico_df['fico_score'] <= boundaries[i + 1])]
            for i in range(num_buckets)
        ]
        ll = compute_log_likelihood(bucketed)
        if ll > best_ll:
            best_ll = ll
            best_boundaries = boundaries

    return best_boundaries, best_ll

def print_buckets(boundaries, label):
    print(f"\n{label} Bucket Ranges:")
    for i in range(len(boundaries) - 1):
        print(f"Bucket {i + 1}: {boundaries[i] + 1} – {boundaries[i + 1]}")

low_buckets, low_ll = find_best_buckets(low_range, 5)
high_buckets, high_ll = find_best_buckets(high_range, 5)

print_buckets(low_buckets, "Low FICO (<= 600)")
print_buckets(high_buckets, "High FICO (> 600)")


Low FICO (<= 600) Bucket Ranges:
Bucket 1: 409 – 538
Bucket 2: 539 – 562
Bucket 3: 563 – 578
Bucket 4: 579 – 590
Bucket 5: 591 – 600

High FICO (> 600) Bucket Ranges:
Bucket 1: 602 – 626
Bucket 2: 627 – 647
Bucket 3: 648 – 671
Bucket 4: 672 – 701
Bucket 5: 702 – 850


In [4]:
all_boundaries = low_buckets[:-1] + high_buckets 

ratings = list(range(10, 0, -1))  

def assign_fico_rating(score):
    for i in range(len(all_boundaries) - 1):
        if all_boundaries[i] < score <= all_boundaries[i + 1]:
            return ratings[i]
    return None 

df['fico_rating'] = df['fico_score'].apply(assign_fico_rating)

print(df[['fico_score', 'fico_rating']].head(10))

   fico_score  fico_rating
0         605          5.0
1         572          8.0
2         602          5.0
3         612          5.0
4         631          4.0
5         697          2.0
6         722          1.0
7         545          9.0
8         676          2.0
9         447         10.0


In [5]:
df.head(10)

,customer_id,credit_lines_outstanding,loan_amt_outstanding,total_debt_outstanding,income,years_employed,fico_score,default,fico_rating
0,8153374,0,5221.545193,3915.471226,78039.38546,5,605,0,5.0
1,7442532,5,1958.928726,8228.752520,26648.43525,2,572,1,8.0
2,2256073,0,3363.009259,2027.830850,65866.71246,4,602,0,5.0
3,4885975,0,4766.648001,2501.730397,74356.88347,5,612,0,5.0
4,4700614,1,1345.827718,1768.826187,23448.32631,6,631,0,4.0
5,4661159,0,5376.886873,7189.121298,85529.84591,2,697,0,2.0
6,8291909,1,3634.057471,7085.980095,68691.57707,6,722,0,1.0
7,4616950,4,3302.172238,13067.570210,50352.16821,3,545,1,9.0
8,3395789,0,2938.325123,1918.404472,53497.37754,4,676,0,2.0
9,4045948,0,5396.366774,5298.824524,92349.55399,2,447,0,10.0


In [6]:
#func to calculate the rating given a fico score
def calcrat(ficoscore):
    if 702 <= ficoscore <= 850:
        return 1
    elif 672 <= ficoscore <= 701:
        return 2
    elif 648 <= ficoscore <= 671:
        return 3
    elif 627 <= ficoscore <= 647:
        return 4
    elif 602 <= ficoscore <= 626:
        return 5
    elif 591 <= ficoscore <= 600:
        return 6
    elif 579 <= ficoscore <= 590:
        return 7
    elif 563 <= ficoscore <= 578:
        return 8
    elif 539 <= ficoscore <= 562:
        return 9
    elif 409 <= ficoscore <= 538:
        return 10
    else:
        return "Can't be rated"

In [7]:
#An example
rat = calcrat(722)
print(rat)

1
